In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklear

In [4]:
# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [5]:


# Define the LSTM Model for time-series prediction
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout=0.0):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Define the LSTM layer (batch_first=True makes input shape: [batch, time_steps, features])
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, dropout=dropout)
        
        # Fully connected layer for final output
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden state and cell state (shape: [num_layers, batch, hidden_size])
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward pass through LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Take the output of the last time step
        out = self.fc(out[:, -1, :])
        return out



In [6]:
# Hyperparameters (adjust these based on your dataset and experimentation)
input_size = 10    # Number of weather features per time step (e.g., temperature, humidity, etc.)
hidden_size = 50   # Number of hidden units (tune as needed)
output_size = 1    # Predicting one value: GHI
num_layers = 2     # Number of LSTM layers
dropout = 0.2      # Optional dropout between LSTM layers



In [7]:
# Example for one time step:
# [sky_layer1, sky_layer2, sky_layer3, sky_layer4, thunderstorm_code,
#  rain_code, drizzle_code, snow_code, snow_showers_code, ice_pellets_code,
#  fog_code, smoke_code]
# Example vector might look like this
time_step_input = [2, 0, 0, 0, 1, 3, 0, 0, 0, 0, 1, 0]

# And if it was trained on the model below it one batch would be of dimention 
(batch_size, sequence_length, num_features) = (32, 5, 12)


In [8]:
# Create the LSTM model instance and move it to the selected device
model = LSTMModel(input_size, hidden_size, output_size, num_layers, dropout).to(device)
print(model)


# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error is common for regression tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Sample Training Loop
num_epochs = 100

for epoch in range(num_epochs):
    # In practice, load your data here. For demonstration, we use random data:
    # Inputs: shape (batch_size, sequence_length, input_size)
    # Targets: shape (batch_size, output_size)
    batch_size = 32
    sequence_length = 5  # Number of time steps per sample eg this is 5 hours I think

    # Replace random weights with a vector of features

    
    inputs = torch.randn(batch_size, sequence_length, input_size).to(device)
    targets = torch.randn(batch_size, output_size).to(device)
    
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



LSTMModel(
  (lstm): LSTM(10, 50, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)
Epoch [10/100], Loss: 0.9708
Epoch [20/100], Loss: 1.0359
Epoch [30/100], Loss: 1.1619
Epoch [40/100], Loss: 1.5881
Epoch [50/100], Loss: 0.9675
Epoch [60/100], Loss: 0.7881
Epoch [70/100], Loss: 1.1445
Epoch [80/100], Loss: 0.7418
Epoch [90/100], Loss: 1.4474
Epoch [100/100], Loss: 0.7708


In [9]:
# Inference example with new data

# Use test data here

model.eval()  # set model to evaluation mode
with torch.no_grad():
    # Example new_data: one sample, sequence length 5
    new_data = torch.randn(1, sequence_length, input_size).to(device)
    predicted_output = model(new_data)
    print("Predicted Output:", predicted_output.cpu().numpy())


Predicted Output: [[0.07490139]]


In [10]:
# Use somthing like this to preproccess the data to make sure it is in the same ranges

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

ModuleNotFoundError: No module named 'sklearn'